# **1.01 - Audio Feature Extraction**

Adding Features to haunted places dataset. 

Features added using Pandas and Regular Expressions.

## **Audio Feature Extraction**

Used keywords specified in **audio_keywords.txt**. 

Keywords generated with 2 queries to chatgpt:
 - "could you generate a long list of words that are similar to "hear", "overheard", "hearing", "sound", in all tenses."
 - "could you generate more words similar to "screams" "crying" "cries" etc"
 
**"Audio Evidence" [bool]**
- True - audio evidence in description
    - example True {index: 2}: "Others report seeing nothing at all but hearing the killer\'s shouts of rage followed by the victims\' screams of agony"
- False - otherwise
    -


### Feature Extraction

In [ ]:
import pandas as pd
import re 
import time


# Reading CSV
df = pd.read_csv("../data/processed/haunted_places_cleaned.tab", sep = "\t")

# Define Audio Keyword List
audio_keywords = open(f'../data/keywords/audio_keywords.txt', 'r').read().split(',')

def contains_audio_keywords(text):
    if isinstance(text, str):
        for keyword in audio_keywords:
            if re.search(keyword, text, re.IGNORECASE):
                return True  
    return False

start = time.time()
df["Audio_Evidence"] = df["description"].apply(contains_audio_keywords)
end = time.time()

df['Audio_Evidence'].value_counts()

      city        country                                        description  \
0      Ada  United States  ada witch sometimes see misty blue figure floa...   
1  Addison  United States  little girl killed suddenly waiting school bus...   
2   Adrian  United States  take gorman rd west towards sand creek come gr...   
3   Adrian  United States  1970 's one room room 211 old section dorms ex...   
4   Albion  United States  kappa delta sorority kappa delta sorority haun...   

                   location     state state_abbrev  longitude   latitude  \
0              Ada Cemetery  Michigan           MI -85.504893  42.962106   
1           North Adams Rd.  Michigan           MI -84.381843  41.971425   
2             Ghost Trestle  Michigan           MI -84.035656  41.904538   
3  Siena Heights University  Michigan           MI -84.017565  41.905712   
4            Albion College  Michigan           MI -84.745177  42.244006   

   city_longitude  city_latitude  
0      -85.495480      42.9

Audio_Evidence
True     5595
False    5397
Name: count, dtype: int64

### Post Processing

In [13]:
counts = df["Audio_Evidence"].value_counts()
true_counts = counts.get(True, 0)
false_counts = counts.get(False, 0)

extract_printout = [
    ("True Counts", true_counts), 
    ("False Counts", false_counts)]

### Report and Save

In [14]:
## Printout Report ##
print("-" * 150, "Extraction Completed", "-" * 150)
print(f"Extraction Took: {end - start:.6f} seconds", end = "\n\n")
print("\n".join([f"{extract_str[0]}: {extract_str[1]}" for extract_str in extract_printout]))
print("-" * 150)

------------------------------------------------------------------------------------------------------------------------------------------------------ Extraction Completed ------------------------------------------------------------------------------------------------------------------------------------------------------
Extraction Took: 1.823930 seconds

True Counts: 5595
False Counts: 5397
------------------------------------------------------------------------------------------------------------------------------------------------------
